## 감성 분석(Keras)
---
Keras
로지스틱 회귀를 활용하여 감성사전을 만들고 감성분석하기

### 1. Keras를 활용한 로지스틱 회귀 분석

#### 아마존 제품 리뷰 TDM 분석 결과 및 원본 파일 불러오기

In [1]:
import joblib

In [2]:
with open('amazon.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)

#### 학습용 / 테스트용 데이터 분할

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(tdm, sentiment, test_size = 0.2, random_state=1234)

In [5]:
print(X_train.shape, X_test.shape)

(800, 1000) (200, 1000)


#### 모델 생성

In [6]:
from keras import Sequential
from keras.layers import Dense, Input

In [7]:
NUM_WORDS = tdm.shape[1]
m1 = Sequential()
m1.add(Dense(1, activation = "sigmoid", input_shape = (NUM_WORDS, )))
m1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 1,001
Trainable params: 1,001
Non-trainable params: 0
_________________________________________________________________


#### 학습 및 평가

In [8]:
from keras.optimizers import Adam

In [9]:
import numpy as np

In [10]:
m1.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=Adam())
X_train = X_train.toarray()               
# 데이터 저장 형식에 문제가 있는지 numpy를 통해 toarray()해야만 정상동작함
m1.fit(X_train, y_train, epochs=30)

Epoch 1/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6921 - accuracy: 0.5213
Epoch 2/30
25/25 [==============================] - 0s 2ms/step - loss: 0.6868 - accuracy: 0.6025
Epoch 3/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6820 - accuracy: 0.6650
Epoch 4/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6773 - accuracy: 0.7200
Epoch 5/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6725 - accuracy: 0.7550
Epoch 6/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6678 - accuracy: 0.7837
Epoch 7/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6632 - accuracy: 0.8138
Epoch 8/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6586 - accuracy: 0.8363
Epoch 9/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6541 - accuracy: 0.8500
Epoch 10/30
25/25 [==============================] - 0s 1ms/step - loss: 0.6496 - accuracy: 0.8550
Epoch 11/30
25/25 [

#### 예측 및 평가

In [18]:
from sklearn.metrics import accuracy_score

In [22]:
# y_pred = m1.predict_classes(X_test) 오류발생. tensorflow v2.6 이후로 predict_classes가 없어짐
# 시그모이드를 사용하는 binary claaification인 경우 아래와 같이 수정
X_test = X_test.toarray()
y_pred = (m1.predict(X_test) > 0.5).astype("int32")

7/7 [==============================] - 0s 1ms/step


In [23]:
accuracy_score(y_test, y_pred)

0.75

### 2. 다층 신경망 생성

#### 신경망 생성 및 학습

In [24]:
m2 = Sequential()
m2.add(Dense(100, activation='relu', input_shape=(NUM_WORDS,)))
m2.add(Dense(1, activation='sigmoid'))
m2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 100)               100100    
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 100,201
Trainable params: 100,201
Non-trainable params: 0
_________________________________________________________________


In [25]:
m2.compile(loss = "binary_crossentropy", metrics=["accuracy"], optimizer = Adam())

In [27]:
m2.fit(X_train, y_train, epochs = 30, verbose = 0)

In [30]:
# y_pred = m1.predict_classes(X_test) 오류발생. tensorflow v2.6 이후로 predict_classes가 없어짐
y_pred2 = (m2.predict(X_test) > 0.5).astype("int32")
accuracy_score(y_test, y_pred2)

7/7 [==============================] - 0s 2ms/step


0.8